# Issue Credential - Bob
## Role: Holder

### This tutorial runs through the issuer-api from the perspective of a holder. It should be run alongside the [issuer notebook](http://localhost:8888/notebooks/Part%205%20-%20Issue%20Credential.ipynb) from Alice's perspective. 


If unfamiliar with the protocol it is worth reading through the [aries-rfs](https://github.com/hyperledger/aries-rfcs/tree/master/features/0036-issue-credential)

## First complete steps 1-6 in the issuer notebook.

## 7. Initialise the Controller

In [ ]:
%autoawait
import time
import asyncio

from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8052
WEBHOOK_BASE = ""
ADMIN_URL = "http://bob-agent:8051"

agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL, connections=True)

## 8. Register listeners

The handler should get called every time the controller receives a webhook with the topic issue_credential, printing out the payload. The agent calls to this webhook every time it receives an issue-credential protocol message from a credential.

In [ ]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())
def cred_handler(payload):
    print("Handle Credentials")
    exchange_id = payload['credential_exchange_id']
    state = payload['state']
    role = payload['role']
    attributes = payload['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {exchange_id}, role: {role}, state: {state}")
    print(f"Attributes: {attributes}")
    
cred_listener = {
    "topic": "issue_credential",
    "handler": cred_handler
}
agent_controller.register_listeners([cred_listener], defaults=True)

## 9. Continue in the [Issuer](http://localhost:8888/notebooks/issuer.ipynb) Notebook

This sends the credential to the holder Bob and should be recognisable through the print statements from the above handler.

## 10. Check Credential Exchange Records

The agent will have at least one record if you have run through the issuer notebook up until send credential.

In [ ]:
response["results"]

[{'state': 'offer_received',
  'credential_exchange_id': 'aa0f675e-a5ba-4bab-8e64-a1729798df3c',
  'updated_at': '2020-12-01 06:36:50.113438Z',
  'thread_id': '4e49e299-5c51-43e5-83ea-ba9089e98428',
  'auto_remove': True,
  'trace': True,
  'credential_proposal_dict': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/issue-credential/1.0/propose-credential',
   '@id': 'b6222a36-f182-4bc0-bc12-e831dbc5eb89',
   'cred_def_id': 'PQRXDxdGqQGSZ8z69p4xZP:3:CL:10:default',
   'credential_proposal': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/issue-credential/1.0/credential-preview',
    'attributes': [{'name': 'name', 'value': 'Bob'},
     {'name': 'skill', 'value': 'researcher'},
     {'name': 'age', 'value': '1337'}]},
   'schema_id': 'PQRXDxdGqQGSZ8z69p4xZP:2:hwrdtm_custom_schema:0.0.1',
   'comment': 'create automated credential exchange'},
  'role': 'holder',
  'schema_id': 'PQRXDxdGqQGSZ8z69p4xZP:2:hwrdtm_custom_schema:0.0.1',
  'connection_id': '6ef9082f-6450-4dfa-99d2-d65cf279d0fc',
 

In [10]:
response = await agent_controller.issuer.get_records()
results = response["results"]
print(len(results))
if len(results) == 0:
    print("You need to first send a credential from the issuer notebook (Alice)")
else:
    cred_record = results[0]
    cred_ex_id = cred_record['credential_exchange_id']
    state = cred_record['state']
    role = cred_record['role']
    attributes = results[0]['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")
    print(f"Being offered: {attributes}")


1
Credential exchange aa0f675e-a5ba-4bab-8e64-a1729798df3c, role: holder, state: offer_received
Being offered: [{'name': 'name', 'value': 'Bob'}, {'name': 'skill', 'value': 'researcher'}, {'name': 'age', 'value': '1337'}]


## 11. Request Credential from Issuer

If happy with the attributes being offered in the credential, then the holder requests the credential from the issuer to proceed with the issuance.

It is only possible to request a credential from an exchange when it is in the offer_received state

In [ ]:
record = await agent_controller.issuer.send_request_for_record(cred_ex_id)
state = record['state']
role = record['role']
print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")

Credential exchange aa0f675e-a5ba-4bab-8e64-a1729798df3c, role: holder, state: request_sent
Handle Credentials
Credential exchange aa0f675e-a5ba-4bab-8e64-a1729798df3c, role: holder, state: request_sent
Attributes: [{'name': 'name', 'value': 'Bob'}, {'name': 'skill', 'value': 'researcher'}, {'name': 'age', 'value': '1337'}]
Handle Credentials
Credential exchange aa0f675e-a5ba-4bab-8e64-a1729798df3c, role: holder, state: credential_received
Attributes: [{'name': 'name', 'value': 'Bob'}, {'name': 'skill', 'value': 'researcher'}, {'name': 'age', 'value': '1337'}]


## 12. Store the credential

Once the issuer has responded to a request by sending the credential, the holder needs to store it to save the credential for later.

First check that the credential record is in the credential_received state

In [ ]:
record = await agent_controller.issuer.get_record_by_id(cred_ex_id)
state = record['state']
role = record['role']
print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")

Credential exchange aa0f675e-a5ba-4bab-8e64-a1729798df3c, role: holder, state: credential_received


In [ ]:
response = await agent_controller.issuer.store_credential(cred_ex_id, "CUSTOM OM Credential")
state = response['state']
role = response['role']
print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")

Credential exchange aa0f675e-a5ba-4bab-8e64-a1729798df3c, role: holder, state: credential_acked
Handle Credentials
Credential exchange aa0f675e-a5ba-4bab-8e64-a1729798df3c, role: holder, state: credential_acked
Attributes: [{'name': 'name', 'value': 'Bob'}, {'name': 'skill', 'value': 'researcher'}, {'name': 'age', 'value': '1337'}]


## Not Included

* Propose Credential: Allows holder to propose an alternative credential that it wishes the issuer to issue. Most likely used when negotiating the values within the credential or if the holder spots an error in current issued credential. **Note: this generates a new credential exchange id.**



## End of Tutorial

Be sure to terminate the controller so you can run another tutorial.

In [ ]:
response = await agent_controller.terminate()


In [15]:
cred_ex_id

'a2536881-8e4d-4c06-a91d-28a2a6439528'